# Учётная запись

Чтобы сгенерировать файл закрытого ключа для вашей учётной записи службы:

1\) В Firebase консоли откройте *Настройки > [Учётные записи служб](https://console.firebase.google.com/project/_/settings/serviceaccounts/adminsdk)*.

2\) Нажмите кнопку *Создать новый секретный ключ*, а затем подтвердить, нажав *Создать ключ*.

3\) Надёжно храните файл JSON, содержащий ключ.

In [1]:
import firebase_admin
from firebase_admin import credentials
import json

In [2]:
SERVICE_ACCOUNT_KEY = '<SERVICE_ACCOUNT_KEY>'

# Cloud Firestore

Более полное руководство здесь: [Начать работу с Cloud Firestore](https://firebase.google.com/docs/firestore/quickstart)

## Подключение к БД

In [3]:
from firebase_admin import firestore

In [4]:
DATABASE_NAME = '<DATABASE_NAME>'

In [5]:
cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
firebase_admin.initialize_app(cred)
db = firestore.client()

## Добавление/обновление данных

Для примера рассмотрим JSON-файл со списком книг. У нас есть информация о четырёх книгах следующим образом.

In [6]:
with open('books.json', 'r') as f:
    file_contents = json.load(f)
file_contents

{'Book1': {'Title': 'The Fellowship of the Ring',
  'Author': 'J.R.R. Tolkien',
  'Genre': 'Epic fantasy',
  'Price': 105,
  'value': 3},
 'Book2': {'Title': 'The Two Towers',
  'Author': 'J.R.R. Tolkien',
  'Genre': 'Epic fantasy',
  'Price': 100,
  'value': 2},
 'Book3': {'Title': 'The Return of the King',
  'Author': 'Tolkien',
  'Genre': 'Fantasy',
  'Price': 80,
  'value': 1},
 'Book4': {'Title': 'Brida',
  'Author': 'Paulo Coelho',
  'Genre': 'Fiction',
  'Price': 90,
  'value': 4}}

Можно сохранить все данные JSON файла целиком в базу дунных.  
В терминах БД:  
 * books - это коллекция  
 * Каждая конкретная книга - это документ.

In [7]:
books_ref = db.collection(u'books')
for key, val in file_contents.items():
    books_ref.document(key).set(val)

Вывод сведений о книгах на экран.

In [8]:
books_ref = db.collection(u'books')
docs = books_ref.stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

Book1 => {'Author': 'J.R.R. Tolkien', 'Price': 105, 'value': 3, 'Title': 'The Fellowship of the Ring', 'Genre': 'Epic fantasy'}
Book2 => {'Genre': 'Epic fantasy', 'Title': 'The Two Towers', 'value': 2, 'Price': 100, 'Author': 'J.R.R. Tolkien'}
Book3 => {'value': 1, 'Price': 80, 'Author': 'Tolkien', 'Genre': 'Fantasy', 'Title': 'The Return of the King'}
Book4 => {'Genre': 'Fiction', 'Title': 'Brida', 'value': 4, 'Price': 90, 'Author': 'Paulo Coelho'}


Обновление данных по книге *'Book1'*.

In [9]:
book_ref = db.collection(u'books').document(u'Book1')
book_ref.update({u'Price': 120})

update_time {
  seconds: 1642831769
  nanos: 506262000
}

Вы можете установить поле в своём документе для отметки времени сервера, которая отслеживает, когда сервер получает обновление.

In [10]:
book_ref = db.collection(u'books').document(u'Book1')
book_ref.update({u'timestamp': firestore.SERVER_TIMESTAMP})

update_time {
  seconds: 1642831769
  nanos: 596514000
}
transform_results {
  timestamp_value {
    seconds: 1642831769
    nanos: 591000000
  }
}

Вы можете увеличивать или уменьшать значение числового поля, как показано в следующем примере. Операция приращения увеличивает или уменьшает текущее значение поля на заданную величину.

In [11]:
book_ref = db.collection(u'books').document(u'Book2')
book_ref.update({'Price': firestore.Increment(50)})

update_time {
  seconds: 1642831769
  nanos: 660887000
}
transform_results {
  integer_value: 150
}

Если документ содержит поле массива, вы можете использовать *arrayUnion()* и *arrayRemove()* для добавления и удаления элементов. *arrayUnion()* добавляет элементы в массив, но только, когда элементы не присутствуют. *arrayRemove()* удаляет все экземпляры каждого данного элемента.

In [12]:
book_ref = db.collection(u'books').document(u'Book2')
book_ref.update({u'pages': firestore.ArrayUnion([12, 15, 20])})

update_time {
  seconds: 1642831769
  nanos: 728670000
}
transform_results {
  null_value: NULL_VALUE
}

In [13]:
book_ref.update({u'pages': firestore.ArrayRemove([15])})

update_time {
  seconds: 1642831769
  nanos: 801748000
}
transform_results {
  null_value: NULL_VALUE
}

## Выборка данных

Извлечь содержимое одного документа.

In [14]:
book_ref = db.collection(u'books').document(u'Book2')

doc = book_ref.get()
if doc.exists:
    print(f'Document data: {doc.to_dict()}')
else:
    print(u'No such document!')

Document data: {'pages': [12, 20], 'Title': 'The Two Towers', 'value': 2, 'Price': 150, 'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy'}


Получить несколько документов из коллекции.

In [15]:
docs = db.collection(u'books').where(u'Price', u'==', 100).stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

Все документы в коллекции.

In [16]:
docs = db.collection(u'books').stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

Book1 => {'value': 3, 'Price': 120, 'Title': 'The Fellowship of the Ring', 'timestamp': DatetimeWithNanoseconds(2022, 1, 22, 6, 9, 29, 591000, tzinfo=datetime.timezone.utc), 'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy'}
Book2 => {'pages': [12, 20], 'Title': 'The Two Towers', 'value': 2, 'Price': 150, 'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy'}
Book3 => {'Author': 'Tolkien', 'Price': 80, 'value': 1, 'Title': 'The Return of the King', 'Genre': 'Fantasy'}
Book4 => {'Genre': 'Fiction', 'Title': 'Brida', 'Price': 90, 'value': 4, 'Author': 'Paulo Coelho'}


Оператор вхождения in проверяет несколько значений.

In [17]:
book_ref = db.collection(u'books')
query = book_ref.where(u'Genre', u'in', [u'Epic fantasy', u'History'])

for doc in query.stream():
    print(f'{doc.id} => {doc.to_dict()}')

Book1 => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'value': 3, 'Title': 'The Fellowship of the Ring', 'Price': 120, 'timestamp': DatetimeWithNanoseconds(2022, 1, 22, 6, 9, 29, 591000, tzinfo=datetime.timezone.utc)}
Book2 => {'value': 2, 'Price': 150, 'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'pages': [12, 20], 'Title': 'The Two Towers'}


Диапазон фильтров только на одном поле.

In [18]:
book_ref = db.collection(u'books')
query = book_ref.where(u'Price', u'>=', 90).where(u'Price', u'<=', 120)

for doc in query.stream():
    print(f'{doc.id} => {doc.to_dict()}')

Book4 => {'Title': 'Brida', 'Genre': 'Fiction', 'Author': 'Paulo Coelho', 'value': 4, 'Price': 90}
Book1 => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'Price': 120, 'value': 3, 'Title': 'The Fellowship of the Ring', 'timestamp': DatetimeWithNanoseconds(2022, 1, 22, 6, 9, 29, 591000, tzinfo=datetime.timezone.utc)}


Составные фильтры на разных полях.

In [19]:
book_ref = db.collection(u'books')
query = book_ref.where(u'Author', u'==', u'J.R.R. Tolkien').where(u'Price', u'==', 120)

for doc in query.stream():
    print(f'{doc.id} => {doc.to_dict()}')

Book1 => {'Price': 120, 'value': 3, 'timestamp': DatetimeWithNanoseconds(2022, 1, 22, 6, 9, 29, 591000, tzinfo=datetime.timezone.utc), 'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'Title': 'The Fellowship of the Ring'}


Для объединения операторов равенства с операторами неравенства, < , <= , > и != вы должны создать [составной индекс](https://firebase.google.com/docs/firestore/query-data/indexing).

Вы можете запросить первые 2 книги в алфавитном порядке по автору.

In [20]:
book_ref = db.collection(u'books')
query = book_ref.order_by(u'Author').limit_to_last(2)
for doc in query.get():
    print(f'{doc.id} => {doc.to_dict()}')

Book2 => {'Title': 'The Two Towers', 'pages': [12, 20], 'Genre': 'Epic fantasy', 'Price': 150, 'value': 2, 'Author': 'J.R.R. Tolkien'}
Book1 => {'Title': 'The Fellowship of the Ring', 'value': 3, 'Price': 120, 'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'timestamp': DatetimeWithNanoseconds(2022, 1, 22, 6, 9, 29, 591000, tzinfo=datetime.timezone.utc)}


In [21]:
book_ref = db.collection(u'books')
query = book_ref.order_by(u'Genre', direction=firestore.Query.DESCENDING).limit(2)
for doc in query.stream():
    print(f'{doc.id} => {doc.to_dict()}')

Book4 => {'Price': 90, 'value': 4, 'Author': 'Paulo Coelho', 'Title': 'Brida', 'Genre': 'Fiction'}
Book3 => {'value': 1, 'Price': 80, 'Author': 'Tolkien', 'Genre': 'Fantasy', 'Title': 'The Return of the King'}


Также можно сделать по нескольким полям. 

In [22]:
book_ref = db.collection(u'books')
query = book_ref.where(u'Price', u'>', 120).order_by(u'Price', direction=firestore.Query.DESCENDING).limit(10)

for doc in query.stream():
    print(f'{doc.id} => {doc.to_dict()}')

Book2 => {'Price': 150, 'value': 2, 'Author': 'J.R.R. Tolkien', 'Title': 'The Two Towers', 'pages': [12, 20], 'Genre': 'Epic fantasy'}


## Удаление данных

Чтобы удалить документ, используйте *delete()* метод.

In [23]:
db.collection(u'books').document(u'Book4').delete()

DatetimeWithNanoseconds(2022, 1, 22, 6, 9, 30, 546915, tzinfo=datetime.timezone.utc)

Для того, чтобы удалить определённые поля из документа, используйте *FieldValue.delete()* метод при обновлении документа.

In [24]:
book_ref = db.collection(u'books').document(u'Book3')
book_ref.update({u'Author': firestore.DELETE_FIELD})

update_time {
  seconds: 1642831770
  nanos: 616314000
}

In [25]:
books_ref = db.collection(u'books')
docs = books_ref.stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

Book1 => {'timestamp': DatetimeWithNanoseconds(2022, 1, 22, 6, 9, 29, 591000, tzinfo=datetime.timezone.utc), 'Genre': 'Epic fantasy', 'Author': 'J.R.R. Tolkien', 'Title': 'The Fellowship of the Ring', 'value': 3, 'Price': 120}
Book2 => {'pages': [12, 20], 'Title': 'The Two Towers', 'Price': 150, 'value': 2, 'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy'}
Book3 => {'Title': 'The Return of the King', 'Genre': 'Fantasy', 'Price': 80, 'value': 1}


In [26]:
firebase_admin.delete_app(firebase_admin.get_app())

# Realtime Database

Более полное руководство здесь: [Начать работу с Realtime Database](https://firebase.google.com/docs/database/admin/start)

## Подключение к БД

In [27]:
from firebase_admin import db

In [28]:
DATABASE_NAME = '<DATABASE_NAME>'
DATABASE_SERVER = '<DATABASE_SERVER>'
DATABASE_URL = f'https://{DATABASE_NAME}-default-rtdb.{DATABASE_SERVER}.firebasedatabase.app'

In [29]:
print(f'Перейти к базе данных: {DATABASE_URL}')

Перейти к базе данных: https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app


Rules:
<code>
{
  "rules": {
    "books" : {
          ".indexOn": ["Price", "Author", "Genre", "Title", "value"]
        },
    ".read": true,
    ".write": true,
  }
}
</code>

In [30]:
cred = credentials.Certificate(SERVICE_ACCOUNT_KEY)
firebase_admin.initialize_app(cred, {'databaseURL': DATABASE_URL})

Мы устанавливаем ссылку к корню базы данных.

In [31]:
ref = db.reference('/')

## Добавление/обновление данных

Все данные базы данных Firebase Realtime хранятся в виде объектов JSON. Вы можете думать о базе данных как о JSON-дереве, размещённом в облаке.

Для примера рассмотрим JSON-файл со списком книг. У нас есть информация о четырёх книгах следующим образом.

In [32]:
with open('books.json', 'r') as f:
    file_contents = json.load(f)
file_contents

{'Book1': {'Title': 'The Fellowship of the Ring',
  'Author': 'J.R.R. Tolkien',
  'Genre': 'Epic fantasy',
  'Price': 105,
  'value': 3},
 'Book2': {'Title': 'The Two Towers',
  'Author': 'J.R.R. Tolkien',
  'Genre': 'Epic fantasy',
  'Price': 100,
  'value': 2},
 'Book3': {'Title': 'The Return of the King',
  'Author': 'Tolkien',
  'Genre': 'Fantasy',
  'Price': 80,
  'value': 1},
 'Book4': {'Title': 'Brida',
  'Author': 'Paulo Coelho',
  'Genre': 'Fiction',
  'Price': 90,
  'value': 4}}

Можно сохранить все данные JSON файла целиком в базу дунных. Сначала обнуляем данную ветвь.

In [33]:
ref = db.reference('/')
ref.set({'books': -1})

Запись JSON целиком.

In [34]:
books_ref = db.reference('/books')
books_ref.set(file_contents)

Запись данных с уникальным идентификатором.

In [35]:
for key, value in file_contents.items():
    books_ref.push().set(value)

Обновление сохранённых данных.

In [36]:
books_ref = db.reference('/books/Book1')
books_ref.update({'Price': 115})

## Выборка данных

Получить все данные.

In [37]:
books_ref = db.reference('/books')
print(f'{DATABASE_URL}/books.json?print=pretty')
books_ref.get()

https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app/books.json?print=pretty


{'-Mu-OrScrUShczL5a9II': {'Author': 'J.R.R. Tolkien',
  'Genre': 'Epic fantasy',
  'Price': 105,
  'Title': 'The Fellowship of the Ring',
  'value': 3},
 '-Mu-OrVRbosP9QezusMv': {'Author': 'J.R.R. Tolkien',
  'Genre': 'Epic fantasy',
  'Price': 100,
  'Title': 'The Two Towers',
  'value': 2},
 '-Mu-OrY0eZm7TXv4hhdm': {'Author': 'Tolkien',
  'Genre': 'Fantasy',
  'Price': 80,
  'Title': 'The Return of the King',
  'value': 1},
 '-Mu-Or_RjSqCDG9NJ-fs': {'Author': 'Paulo Coelho',
  'Genre': 'Fiction',
  'Price': 90,
  'Title': 'Brida',
  'value': 4},
 'Book1': {'Author': 'J.R.R. Tolkien',
  'Genre': 'Epic fantasy',
  'Price': 115,
  'Title': 'The Fellowship of the Ring',
  'value': 3},
 'Book2': {'Author': 'J.R.R. Tolkien',
  'Genre': 'Epic fantasy',
  'Price': 100,
  'Title': 'The Two Towers',
  'value': 2},
 'Book3': {'Author': 'Tolkien',
  'Genre': 'Fantasy',
  'Price': 80,
  'Title': 'The Return of the King',
  'value': 1},
 'Book4': {'Author': 'Paulo Coelho',
  'Genre': 'Fiction',
  

Изменение цены книги по одному автору.

In [38]:
best_sellers = books_ref.get()
for key, value in best_sellers.items():
    if(value['Author'] == 'Tolkien'):
        books_ref.child(key).update({'Price': 75})

Упорядочевание по значению цены.

In [39]:
snapshot = books_ref.order_by_child('Price').get()
print(f'{DATABASE_URL}/books.json?orderBy="Price"&print=pretty')
for key, val in snapshot.items():
    print(f'{key} => {val}')

https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app/books.json?orderBy="Price"&print=pretty
-Mu-OrY0eZm7TXv4hhdm => {'Author': 'Tolkien', 'Genre': 'Fantasy', 'Price': 75, 'Title': 'The Return of the King', 'value': 1}
Book3 => {'Author': 'Tolkien', 'Genre': 'Fantasy', 'Price': 75, 'Title': 'The Return of the King', 'value': 1}
-Mu-Or_RjSqCDG9NJ-fs => {'Author': 'Paulo Coelho', 'Genre': 'Fiction', 'Price': 90, 'Title': 'Brida', 'value': 4}
Book4 => {'Author': 'Paulo Coelho', 'Genre': 'Fiction', 'Price': 90, 'Title': 'Brida', 'value': 4}
-Mu-OrVRbosP9QezusMv => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'Price': 100, 'Title': 'The Two Towers', 'value': 2}
Book2 => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'Price': 100, 'Title': 'The Two Towers', 'value': 2}
-Mu-OrScrUShczL5a9II => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'Price': 105, 'Title': 'The Fellowship of the Ring', 'value': 3}
Book1 => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic f

Можно упорядочить узлы по их ключам.

In [40]:
snapshot = books_ref.order_by_key().start_at('Book1').end_at('Book2').get()
print(f'{DATABASE_URL}/books.json?orderBy="$key"&startAt="Book1"&endAt="Book2"&print=pretty')
for key in snapshot:
    print(key)

https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app/books.json?orderBy="$key"&startAt="Book1"&endAt="Book2"&print=pretty
Book1
Book2


Самая дорогая книга.

In [41]:
snapshot = books_ref.order_by_child('Price').limit_to_last(1).get()
print(f'{DATABASE_URL}/books.json?orderBy="Price"&limitToLast=1&print=pretty')
for key, val in snapshot.items():
    print(f'{key} => {val}')

https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app/books.json?orderBy="Price"&limitToLast=1&print=pretty
Book1 => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'Price': 115, 'Title': 'The Fellowship of the Ring', 'value': 3}


Самая дешёвая книга.

In [42]:
snapshot = books_ref.order_by_child('Price').limit_to_first(1).get()
print(f'{DATABASE_URL}/books.json?orderBy="Price"&limitToFirst=1&print=pretty')
for key, val in snapshot.items():
    print(f'{key} => {val}')

https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app/books.json?orderBy="Price"&limitToFirst=1&print=pretty
-Mu-OrY0eZm7TXv4hhdm => {'Author': 'Tolkien', 'Genre': 'Fantasy', 'Price': 75, 'Title': 'The Return of the King', 'value': 1}


Отбор по значению цены.

In [43]:
print(f'{DATABASE_URL}/books.json?orderBy="Price"&equalTo=100&print=pretty')
books_ref.order_by_child('Price').equal_to(100).get()

https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app/books.json?orderBy="Price"&equalTo=100&print=pretty


OrderedDict([('-Mu-OrVRbosP9QezusMv',
              {'Author': 'J.R.R. Tolkien',
               'Genre': 'Epic fantasy',
               'Price': 100,
               'Title': 'The Two Towers',
               'value': 2}),
             ('Book2',
              {'Author': 'J.R.R. Tolkien',
               'Genre': 'Epic fantasy',
               'Price': 100,
               'Title': 'The Two Towers',
               'value': 2})])

Цена книги по крайней мере 105.

In [44]:
snapshot = books_ref.order_by_child('Price').start_at(105).get()
print(f'{DATABASE_URL}/books.json?orderBy="Price"&startAt=105&print=pretty')
for key, val in snapshot.items():
    print(f'{key} => {val}')

https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app/books.json?orderBy="Price"&startAt=105&print=pretty
-Mu-OrScrUShczL5a9II => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'Price': 105, 'Title': 'The Fellowship of the Ring', 'value': 3}
Book1 => {'Author': 'J.R.R. Tolkien', 'Genre': 'Epic fantasy', 'Price': 115, 'Title': 'The Fellowship of the Ring', 'value': 3}


Выводит все узлы до *'Book1'* лексиграфически.

In [45]:
snapshot = books_ref.order_by_key().end_at('Book1').get()
print(f'{DATABASE_URL}/books.json?orderBy="$key"&endAt="Book1"&print=pretty')
for key in snapshot:
    print(key)

https://test-3835c-default-rtdb.europe-west1.firebasedatabase.app/books.json?orderBy="$key"&endAt="Book1"&print=pretty
-Mu-OrScrUShczL5a9II
-Mu-OrVRbosP9QezusMv
-Mu-OrY0eZm7TXv4hhdm
-Mu-Or_RjSqCDG9NJ-fs
Book1


## Удаление данных

Удаляем одну книгу.

In [46]:
books_ref = db.reference('/books')
books_ref.update({'Book1': None})

In [47]:
for key in books_ref.get():
    print(key)

-Mu-OrScrUShczL5a9II
-Mu-OrVRbosP9QezusMv
-Mu-OrY0eZm7TXv4hhdm
-Mu-Or_RjSqCDG9NJ-fs
Book2
Book3
Book4


Очищаем все данные.

In [48]:
ref = db.reference('/')
ref.set({'books': -1})

In [49]:
firebase_admin.delete_app(firebase_admin.get_app())